# Analysis via zoom into protostars and cutouts of the N$_2$H$^+$ cube
The same we did for HC3N but we separate the files to keep order

In [1]:
import numpy as np
import os
import astropy.units as u
from astropy.wcs import WCS
from astropy.io import fits
from astropy.coordinates import SkyCoord, FK5
from astropy.nddata.utils import Cutout2D
from spectral_cube import SpectralCube
import pandas as pd
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import cm
%matplotlib widget

sourcecat = '../../data/my_catalog_vel.csv'
cubefile = '../../data/NGC1333-N2Hp_match_kms_F1F_01_12.fits'
mlex1file = '../../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x1_filtered_QA.fits'
mlex2file = '../../bayes_frame/nested-sampling/N2Hp/NGC1333-SE-mle-x2_filtered_QA.fits'
#I think in some cases we also need the clusters
cut_clusters = True
clusters_n2hp_files = sorted(glob('../../clustering/cluster*[0-9]_DBSCAN_N2Hp.fits'))


radiuscut_au = 10000
distance = 298 # pc
radiuscut = radiuscut_au / distance *u.arcsec # 

In [2]:
cube = SpectralCube.read(cubefile)
wcsskycube = WCS(cube.header).celestial

mlex1, mlex1head = fits.getdata(mlex1file, header=True)
mlex2, mlex2head = fits.getdata(mlex2file, header=True)

if cut_clusters:
    cluster_n2hp_master = [fits.getdata(filein) for filein in clusters_n2hp_files]
    cluster_head = fits.getheader(clusters_n2hp_files[0])

catalog_protostars = pd.read_csv(sourcecat)


In [3]:
sourcename = 'SK15'
racent, deccent = catalog_protostars.loc[15, ['RAJ2000', 'DEJ2000']].values
subcubes_basename = sourcename + '_N2Hp_{}'
vlsr = catalog_protostars.loc[15, 'vlsr']

In [4]:
# here we cut the images
centerx, centery = wcsskycube.all_world2pix([racent], [deccent], 0)
centerx, centery = int(centerx[0]), int(centery[0])

radiuspix = int(np.round(radiuscut.to(u.deg).value / np.abs(cube.header['CDELT2']), 0))

In [5]:
if not os.path.exists(subcubes_basename.format('cube.fits')):
    subcube = cube[:, centery-radiuspix:centery+radiuspix, centerx-radiuspix:centerx+radiuspix]
    subcube.write(subcubes_basename.format('cube.fits'))
else:
    subcube = SpectralCube.read(subcubes_basename.format('cube.fits'))
headersmall = subcube.header
headersmall_mle = WCS(headersmall).celestial.to_header()
headersmall_mle['NAXIS'] = 3
headersmall_mle['BUNIT'] = 'km s-1'
headersmall_mle['BMAJ'] = headersmall['BMAJ']
headersmall_mle['BMIN'] = headersmall['BMIN']
headersmall_mle['BPA'] = headersmall['BPA']
headersmall_mle['RESTFREQ'] = headersmall['RESTFREQ']

In [6]:

if not os.path.exists(subcubes_basename.format('mlex1.fits')):
    submlex1 = mlex1[:, centery-radiuspix:centery+radiuspix, centerx-radiuspix:centerx+radiuspix]
    headersmall_mle['NAXIS3'] = 6
    fits.writeto(subcubes_basename.format('mlex1.fits'), submlex1, headersmall_mle)
else:
    submlex1, headersmall_mle = fits.getdata(subcubes_basename.format('mlex1.fits'), header=True)

if not os.path.exists(subcubes_basename.format('mlex2.fits')):
    submlex2 = mlex2[:, centery-radiuspix:centery+radiuspix, centerx-radiuspix:centerx+radiuspix]
    headersmall_mle['NAXIS3'] = 12
    fits.writeto(subcubes_basename.format('mlex2.fits'), submlex2, headersmall_mle)
else:
    submlex2, headersmall_mle = fits.getdata(subcubes_basename.format('mlex2.fits'), header=True) 


In [7]:
if cut_clusters:
    subclusters_master = []
    for i, cluster in enumerate(cluster_n2hp_master):
        if not os.path.exists(subcubes_basename.format('cluster{}.fits'.format(i))):
            subcluster = cluster[:, centery-radiuspix:centery+radiuspix, centerx-radiuspix:centerx+radiuspix]
            if np.isnan(subcluster).all(): continue
            else:
                subclusters_master.append(subcluster)
                headersmall_mle['NAXIS3'] = 6
                fits.writeto(subcubes_basename.format('cluster{}.fits'.format(i)), subcluster, headersmall_mle)
        else:
            subclusters_master.append(fits.getdata(subcubes_basename.format('cluster{}.fits'.format(i))))

## Exploration plots